<h2>Web scrapping EPL Stats Website</h2>

https://fbref.com offers all football related stats - match stats, club stats, player stats.
The website contains all the data in tabular form which is easy to scrape.

The goal of this task is to scrape the website for English Football statistics of the last 2 seasons. The statistics must contain match statistics of each club going head to head across all matches in the Premier League, FA Cup, Community Shield and Carabao Cup. 

The task is divided into 2 parts:
    <ul>
        <li> Sample code on how we will scrape the website. For this we will take an example of one club
        <li> Our main script to scrape all the required data from the website, and save it as a csv file locally.
    </ul>
    
<hr>

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time

<h3> Sample code on scrapping the website </h3>
<hr>
The page - https://fbref.com/en/comps/9/Premier-League-Stats, provides us with the list of all the Premier league teams. Querying the page's HTML shows us there is a table containing the current Premier League standings and a hyperlink to the stats of each team.
<hr>

In [24]:
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(url)
data.text[:1000]

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/202205101" />\n    <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://quantcast.mgr.consensu.org\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js\')\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insert

<hr>

The standings table has the class name `stats_table`. Using beautifulSoup, we select the table, and find all the anchor tags (`<a>`) within and get the `href` property. The Urls to the individual teams look like this - https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats, here we notice that `/squads/` route in the URL is common across all links for the team stats.

Filtering the links on `/squads/` and adding the absolute path, we get the 20 team stat links.
<hr>

In [4]:
soup = BeautifulSoup(data.text)
plTable = soup.select('table.stats_table')[0]
links = plTable.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/33c895d4/Southampton-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.

<hr>
For the example, we select the first link of Manchester City, and extract the head to head stats for the current season.

On the link extracted from the previous step, we make a get call to retrieve the page HTML. The table `Scores & Fixtures` contains the head to head stats. Pandas offers the `read_html()` method which can extract a table from HTML page, given the Table name or caption.
<hr>

In [5]:
man_city = team_urls[0]
team_data = requests.get(man_city)
matches = pd.read_html(team_data.text, match = 'Scores & Fixtures')[0]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN


<hr>
In the table retrieved we can observe some basic stats like the scoreline, formation, possession, referees and captain of the match. To analyse the match better, we require more granular and in-depth stats on various aspects of the game. On the same page, there are tabs (Shooting, Passing, Goalkeeping, Defensive Actions etc.) providing in-depth stats.

First, we have to extract these links, and retrieve the HTML for these pages. The shooting stats URL is of this kind - https://fbref.com//en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions, here we see the keyword in the URL is `/all_comps/shooting`. After extracting all the anchor tags (`<a>`) and the `href` attribute, we filter on the above keyword for a list of links.
<hr>

In [6]:
soup = BeautifulSoup(team_data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
team_shooting_link = [f"https://fbref.com/{l}" for l in links if l and "all_comps/shooting" in l][1]
team_shooting_link

'https://fbref.com//en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions'

<hr>
We execute a get request on the link above and retrieve the HTML. The table we need has the header 'Shooting'. Thus we call the `read_html()` function to match on the header.

The columns of the table are multi-level. Multi-level columns are not very useful for analysis and hence we drop the top level.
<hr>

In [25]:
shooting_data = requests.get(team_shooting_link)
man_city_shooting = pd.read_html(shooting_data.text, match= "Shooting")[0]
man_city_shooting.columns = man_city_shooting.columns.droplevel()
man_city_shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,16.9,1.0,0,0,1.9,1.9,0.11,-1.9,-1.9,Match Report
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,17.3,1.0,0,0,2.7,2.7,0.17,1.3,1.3,Match Report
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,14.3,0.0,0,0,3.8,3.8,0.15,1.2,1.2,Match Report
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,14.0,0.0,0,0,2.9,2.9,0.12,-1.9,-1.9,Match Report


<hr>
From the list of all columns, we only need some shooting stats. We will ignore the expected ones ('xG') as this match has already occured and we need absolute stats of what has happened in the game.

We merge the extracted table to our main stats table retrieved earlier. The column to be merged on is the Date, as it is common across all the tables and same for each match.
<hr>

In [95]:
shooting_columns = ['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']
man_city_team = matches.merge(man_city_shooting[shooting_columns], on='Date')
man_city_team.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,12,3,NaN,NaN,0,0
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,4-3-3,Anthony Taylor,Match Report,NaN,18,4,16.9,1.0,0,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,4-3-3,Graham Scott,Match Report,NaN,16,4,17.3,1.0,0,0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,4-3-3,Martin Atkinson,Match Report,NaN,25,10,14.3,0.0,0,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,25,8,14.0,0.0,0,0


<hr>
<hr>

<h3>Code to build dataset</h3>
<hr>

The code below applies a loop over the seasons we want to scrape. For this task we are scrapping data for the 2 seasons, current and last season. For each team we want the head-to-head stats including shooting, passing, goalkeeping, defensive stats.

In [9]:
# Initializing the tables we want to extract and the columns from those tables
stat_substrings = ['shooting', 'keeper', 'passing_types', 'misc']
table_headers = ['Shooting', 'Goalkeeping', 'Pass Types', 'Miscellaneous Stats']
stat_columns = [
    ["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"],
    ['Date', 'Saves', '#OPA', 'SoTA'],
    ['Date', 'Att', 'Cmp', 'Crs', 'Off'],
    ['Date', 'CrdY', 'CrdR', 'Fls', 'Fld', 'OG', 'Recov', 'Int']
]

# Store extracted data of each team as elements of the list
all_matches = []

# Seasons to be scrapped
years = list(range(2022,2020,-1))
years

[2022, 2021]

In [ ]:
# URL of the webpage
url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    
    # Extract data of the page, and retrieve the stats_table -> this will give us the standings and links for teams
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    pl_table = soup.select('table.stats_table')[0]
    
    # Extract and create absolute links for each individual team from the stats_table
    links = [link.get("href") for link in pl_table.find_all('a')]
    team_urls = [f"https://fbref.com/{l}" for l in links if '/squads/' in l]
    
    # Update the main URL of the webpage to URL for the previous season -> This will be applicable for the next iteration
    previous_season = soup.select('a.prev')[0].get("href")
    url = f"https://fbref.com{previous_season}"
    
    # Loop over each team in the standings
    for team_url in team_urls:
        
        # Manipulate string to get Team name -> This will be used to identify data points in the dataset
        team_name = team_url.split('/')[-1].replace('-Stats','').replace('-',' ')
        
        try:
            time.sleep(2)
            
            # For each team, extract the page HTML and read the Scores & Fixtures table
            data = requests.get(team_url)
            matches = pd.read_html(data.text, match= 'Scores & Fixtures')[0]
            
            # Extract links for in-depth stats tables
            soup = BeautifulSoup(data.text)
            links = [l.get('href') for l in soup.find_all('a')]
        
            for substr, hdr, cols in zip(stat_substrings, table_headers, stat_columns):
                
                # Retrieve the page HTML for each category of stats
                stat_link = [f"https://fbref.com/{l}" for l in links if l and 'all_comps/'+substr+'/' in l]
                data = requests.get(stat_link[0])

                try:
                    
                    # Read the table, filter on the columns and merge to the existing dataframe
                    tmp_df = pd.read_html(data.text, match= hdr)[0]
                    tmp_df.columns = tmp_df.columns.droplevel()
                    matches = matches.merge(tmp_df[cols], on='Date')
                    
                except ValueError:
                    continue
                    
        except ValueError:
            continue
        
        # Add columns to each team table for identification of team and season
        matches['Season'] = year
        matches['Team'] = team_name
        all_matches.append(matches)

    time.sleep(30)

# Concat all the tables from the list and save locally to a csv file.
df = pd.concat(all_matches)
df.to_csv('PL_Teams_Datav2.csv')

<hr>
The columns extracted from the webpage have short forms which are explained on the page. We will rename those columns to increase readability and understanding.
<hr>

In [16]:
columns_to_rename = {
    'Comp': 'Competition',
    'Poss': 'Possession',
    'Sh': 'Total_Shots',
    'SoT': 'Shots_on_Target',
    'Dist': 'Avg_distance_Goal',
    'FK': 'FreeKicks',
    'PK': 'PenaltyKicks',
    'PKatt': 'PenaltyKicks_Attempted',
    'SoTA': 'Shots_on_Target_Against',
    '#OPA': 'No_DefensiveActions_Outside_PenaltyBox',
    'Att': 'Passes_Attempted',
    'Cmp': 'Passes_Completed',
    'Crs': 'Croses',
    'Off': 'Offside',
    'CrdY': 'Yellow_cards',
    'CrdR': 'Red_cards',
    'Fls': 'Fouls_Committed',
    'Fld': 'Fouls_Drawn',
    'Recov': 'Ball_Recoveries',
    'Int': 'Interceptions'
}

df.rename(columns= columns_to_rename, inplace= True)

<hr>
The final dataframe extracted from the website looks as below.
<hr>

In [20]:
df.head(5)

,Date,Time,Competition,Round,Day,Venue,Result,GF,GA,Opponent,...,Offside,Yellow_cards,Red_cards,Fouls_Committed,Fouls_Drawn,OG,Ball_Recoveries,Interceptions,Season,Team
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,NaN,2.0,0.0,8.0,6.0,0.0,NaN,3.0,2022,Manchester City
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,0.0,1.0,0.0,12.0,12.0,0.0,90.0,6.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,1.0,1.0,0.0,14.0,6.0,0.0,71.0,10.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,2.0,1.0,0.0,6.0,7.0,0.0,71.0,11.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,4.0,2.0,0.0,15.0,6.0,0.0,79.0,12.0,2022,Manchester City


<hr>